<a href="https://colab.research.google.com/github/pragmatizt/coffee_challenges/blob/master/Another_Earth_Engine_Walkthrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This tutorial came from:** https://colab.research.google.com/github/csaybar/EEwPython/blob/dev/index.ipynb#scrollTo=jgutoC1MiMML

In [1]:
!pip install earthengine-api

In [0]:
!earthengine authenticate

In [0]:
# Let's authenticate Google Drive
"""I've removed this cell and the next, authenticating to Google Drive and Google Cloud doesn't seem necessary"""

# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
# Authenticate to Google cloud
# from google.colab import auth

# auth.authenticate_user()

In [0]:
# Once we've authorized access, initialize connection below:
"""This one I kept, because we definitely do need to initialize ee"""

import ee
ee.Initialize()

## Hello World!


*   **assetID**: is the key to identify a specific file in [GEE datasets](https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003).
*   **ee.Image('assetID')**: In GEE raster data are represented as Image objects (see the next topic).
*   **ee.Imaeg.clip(ee.Geometry.____)**: Is a ee.Image method used to crop spatially bands in Images.
*   **folium**: bridge between python and [Leaflet](https://leafletjs.com/)





In [6]:
import folium

Peru = ee.Image("USGS/SRTMGL1_003")\
          .clip(ee.Geometry.Rectangle(-85, -20, -65, 0))

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

# Use folium to visualzie the imagery.
mapid = Peru.getMapId({'pallete': ['black', 'white'], 'min': 0, 'max': 3000})

map = folium.Map(location=[-10, -75], zoom_start=5)
folium.TileLayer(
    tiles=EE_TILES.format(**mapid),
    attr='Google Earth Engine',
    overlay=True,
    name='median composite',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

## Fundamental Earth Engine data structures

The two most fundamental geographic data structures in Earth Engine are ```Image``` and ```Feature``` corresponding to raster and vector data types, respectively.  Images are composed of bands and a dictionary of properties.  Features are composed of a ```Geometry``` and a dictionary of properties.  A stack of images (e.g. an image time series) is handled by an ```ImageCollection```. A collection of features is handled by a ```FeatureCollection```.  Other fundamental data structures in Earth Engine include:
*   ``Dictionary``
*   ``List``
*   ``Array``
*   ``Geometry``
*   ``Date``
*   ``Number``
*   ``String``

It is important to remember that these are all server-side objects (or containers).  That is, your client browser does not know anything about the objects in your script unless you explicitly request information about them.  That request triggers a message being passed from Google to the Python API.  If the message is large, there will be a corresponding slow down.  Existing two ways to inspect Earth Engine containers in Python console.  The first one is ``print()`` that return the petition (as a JSON) to the server and ``*.getInfo()`` that returns the contents of the ``container``.  The next section summaries all the Data Structures mentioned above.








### Strings
Define a Python string, then put it into the ```ee.String()``` container to be sent to Earth Engine:

In [7]:
# Define a string, then put it into the EE container
aString = 'To the cloud!'
eeString = ee.String(aString)
print('Where to?', eeString)
eeString

Where to? ee.String("To the cloud!")


Think of ```ee.Thing``` as a container for a thing that exists on the server.  In this example, the string is defined first, then put into the container.

### Numbers
Use ```ee.Number()``` to create number objects on the server.

In [8]:
# Define a number that exists on the server.
import numpy as np
serverNumber = ee.Number(np.e)
print('e=', serverNumber)
serverNumber

e= ee.Number(2.718281828459045)


### Lists
To make a Python list into an ```ee.List``` object on the server, you can put a Python literal into a container as with numbers and strings.  Earth Engine also provides server-side convenience methods for making sequences of numbers.

In [9]:
# Make a sequence the hard way.
eeList = ee.List([1, 2, 3, 4, 5])

# Make a sequence the easy way!
sequence = ee.List.sequence(1, 5);

print('Sequence', sequence)
print('Opening in the container:', sequence.getInfo())

Sequence ee.List({
  "type": "Invocation",
  "arguments": {
    "start": 1,
    "end": 5
  },
  "functionName": "List.sequence"
})
Opening in the container: [1, 2, 3, 4, 5]


### Dictionaries
YOu can construct an Earth Engine ```Dictionary``` from a Python object, as with strings, numbers, and lists.  At construction time, you can use Python functionality to initialize the Earth Engine object.  In this case an ee.Dictionary is constructed directly from a Python literal object:

In [10]:
dictionary = ee.Dictionary({
    'e': np.e,
    'pi': np.pi,
    'phi': (1 + np.sqrt(5))/2
})

# Get some values from the dictionary
print('Euler:', dictionary.get('e').getInfo())
print('Pi:', dictionary.get('pi').getInfo())
print('Golden ratio:', dictionary.get('phi').getInfo())

# Get all the keys
print('Keys: ', dictionary.keys().getInfo())

Euler: 2.718281828459045
Pi: 3.141592653589793
Golden ratio: 1.618033988749895
Keys:  ['e', 'phi', 'pi']


## Dates
Date objects are the way Earth Engine represents time. As in the previou examples, it is improtant to distinguish between a Python Date object and an Earth Engine ```ee.Date``` object.  

A date object can be construct from:
*   A string.
*   Python datetime object.
*   Static methods provided by the ee.Date class.

The next examples illustrates the construction of dates from these three forms.   

In [11]:
# Define a date in Earth Engine.
date = ee.Date('2015-12-31')
print('Date:', date)
print('Date:', date.getInfo())

Date: ee.Date({
  "type": "Invocation",
  "arguments": {
    "value": "2015-12-31"
  },
  "functionName": "Date"
})
Date: {'type': 'Date', 'value': 1451520000000}


In earth engine the date is representing with timestamps since midnight on January 1, 1970 considering an in milliseconds timestep.

In [12]:
# Get the current time using the javascript date.now() method.
import datetime as dt
now = dt.datetime.now()
print('milliseconds since January 1, 1970', now)

# Initialize an ee.Date object.
eeNow = ee.Date(now)
print('Now:', eeNow)

milliseconds since January 1, 1970 2020-03-05 05:24:59.366450
Now: ee.Date({
  "type": "Invocation",
  "arguments": {
    "value": 1583385899366
  },
  "functionName": "Date"
})


Arguments to Earth Engine methods can be passed in order, for example to create an ``ee.Date`` from year, month, and day, you can pass paremeters of the ``fromYMD()`` static method in the order year, month, day:

In [13]:
aDate = ee.Date.fromYMD(2017, 1, 13)
print('aDate:', aDate)

aDate: ee.Date({
  "type": "Invocation",
  "arguments": {
    "year": 2017,
    "month": 1,
    "day": 13
  },
  "functionName": "Date.fromYMD"
})


# Next up, image searches.  
Here's the link for this section:
https://colab.research.google.com/github/csaybar/EEwPython/blob/dev/2_eeImage.ipynb

And here's a link to the table of contents of the series:
https://colab.research.google.com/github/csaybar/EEwPython/blob/dev/index.ipynb#scrollTo=jgutoC1MiMML